In [7]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np

# dated='2022-12-26'
from datetime import datetime,timedelta
thedate=datetime.strftime(datetime.today()-timedelta(days=1),"%Y-%m-%d")

df=pd.read_csv(r'DataSource\BaiduYiQingIndex\baidu_yiqing_%s.csv' % thedate)

def convertDate(s):
    s=f'{s:.2f}'.replace('.','/')
    if '12/' in s:
        return '2022/'+s
    else:
        return '2023/'+s

df['date_new']=df['date'].apply(convertDate)
#TG daily wkbk update

tablehead=pd.read_hdf('Processed_Data/bd_covid_index.h5',key='table_head')
c1=df['type']=='百度疫情搜索指数'
c2=df['date_new']>="2023/1/05"
df2=df[c1 & c2]
df3=pd.pivot_table(df2,index='city',columns='date_new',aggfunc='mean',values='index').reset_index()
show=tablehead.merge(df3,on='city',how='left')
show.drop(columns='city').to_clipboard(index=False)

print(thedate)


2023-01-09


In [4]:
c1=df['type']=='百度疫情搜索指数'
ci=df[c1][['city','date_new','index']].rename(columns={'date_new':'date'})
max=ci.groupby('city')['index'].max().reset_index()
max=max.merge(ci,on=['city','index']).rename(columns={'index':'max','date':'peak'})
ci=ci.merge(max,on='city',how='left')
ci['progress_2']=(ci['index']/ci['max'])*100
ci=ci.sort_values(by=['city','date'])
ci=ci.groupby(by='city').tail(1)
ci[['city','peak','progress_2']].to_hdf('Processed_Data/Baidu_covid_progress.h5',key='99city')

In [19]:
# feverindex=df[c1][['city','date_new','index']]
# feverindex.columns=['城市','date','BDfever']
# feverindex.to_hdf(r'Processed_Data/BaiduFever.h5',key='1220')